# Cat Face Localization 

## Mounting to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Check Colab GPU Information

In [ ]:
!nvidia-smi

## Downloading dataset

The original dataset can be found here. In the scope of this exercise, we use its cleaned version. The dataset contains:
* train.csv : .csv file of $8996$ rows, containing image names and coordinates of cat faces in format (x0, y0, width, height)
* test.csv : .csv file of $1000$ rows, containing image names
* images/ : a folder contains $9996$ images with size of $256 \times 256$

In [ ]:
import os
import random

import cv2

import numpy as np
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

In [ ]:
TRAIN_FILE = "/content/drive/My Drive/cat_face_exercise/train.csv"
TEST_FILE = "/content/drive/My Drive/cat_face_exercise/test.csv"
IMAGE_DIR = "/content/drive/My Drive/cat_face_exercise/images"

In [ ]:
df = pd.read_csv(TRAIN_FILE)
df.head()

In [ ]:
test_df = pd.read_csv(TEST_FILE)
test_df.head()

In [ ]:
train_df, val_df = train_test_split(df)

In [ ]:
print(len(df))
print(len(train_df))
print(len(val_df))
print(len(test_df))

In [ ]:
class Config:
    
    seed = 2020
    
    img_width = 256
    img_height = 256
    
    num_classes = 4

    batch_size = 32
    epochs = 2    
    lr = 1e-5
    
    verbose = 1
    
    best_checkpoint_path = 'best.h5'
    latest_checkpoint_path = 'latest.h5'
    
config = Config()   

## TODO 1: Data Loader
 You need to do the augmentation when loading data

In [ ]:
class ImageGenerator:
    
    def create(image_df, augument=True, is_train=True):
        while True:
            
            if is_train:
                image_df = shuffle(image_df, random_state=config.seed)
                
            for start in range(0, len(image_df), config.batch_size):
                end = min(start + config.batch_size, len(image_df))
                batch_images = []
                X_train_batch = image_df.iloc[start:end]
                
                if is_train:
                    batch_labels = np.zeros((len(X_train_batch), 4))
                
                for i in range(len(X_train_batch)):
                    # load image
                    image_path = os.path.join(IMAGE_DIR, X_train_batch.iloc[i]['ImageId'])
                    image = cv2.imread(image_path)
                    
                    if is_train:
                        x0 = X_train_batch.iloc[i]['x0']
                        y0 = X_train_batch.iloc[i]['y0']
                        w = X_train_batch.iloc[i]['width']
                        h = X_train_batch.iloc[i]['height']
                    
                        # augment data
                        if augument:
                            # TODO 1: augmentation
                            pass

                        batch_labels[i][0] = y0 / 256
                        batch_labels[i][1] = x0 / 256
                        batch_labels[i][2] = h / 256
                        batch_labels[i][3] = w / 256

                    batch_images.append(image / 255)
                    
                if is_train:
                    yield np.array(batch_images, np.float32), batch_labels
                else:
                    yield np.array(batch_images, np.float32)

In [ ]:
train_generator = ImageGenerator.create(
    train_df,
    augument=True
)

val_generator = ImageGenerator.create(
    val_df,
    augument=False
)

## TODO 2: Build your own model

In [ ]:
base_model = tf.keras.applications.DenseNet121(include_top=False,weights="imagenet",input_shape=(224,224,3))
base_model.trainable = False
model = tf.keras.Sequential([base_model,tf.keras.layers.GlobalAveragePooling2D(),tf.keras.layers.Dense(4,activation='softmax')])

# Print out model summary
model.summary()

In [ ]:
model.compile(
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(lr=config.lr)
)

In [ ]:
best_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    config.best_checkpoint_path, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False,
    mode='min'
    
)

latest_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    config.latest_checkpoint_path, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=False, 
    save_weights_only=False,
    mode='min'
    
)

early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    patience=10
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    patience=2,
    factor=0.2,
    verbose=1,
    min_lr=1e-9
)

callbacks_list = [
    best_checkpoint,
    latest_checkpoint,
    reduce_lr,
    early
]

## Training

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch= np.ceil(float(len(train_df)) / config.batch_size),
    validation_data=val_generator,
    validation_steps= np.ceil(float(len(val_df)) / config.batch_size),
    epochs=config.epochs,
    callbacks=callbacks_list,
    verbose=config.verbose
)

## Prediction

In [ ]:
test_generator = ImageGenerator.create(
    test_df,
    augument=False,
    is_train=False
)

In [ ]:
ls

In [ ]:
pred_model = tf.keras.models.load_model(config.best_checkpoint_path)

In [ ]:
predict = pred_model.predict(test_generator,
                             steps=np.ceil(float(len(test_df)) / config.batch_size),
                             verbose=config.verbose)

In [ ]:
predict

In [ ]:
predict = predict*256
predict = predict.astype(int)

In [ ]:
print(predict)

## Create submission file

In [ ]:
def clip(val, minval, maxval):
    """Clips a value between min and max (both including)."""
    if val < minval:
        return minval
    elif val > maxval:
        return maxval
    else:
        return val

In [ ]:
# clip coordinates to be inside image
for box in predict:
    y0, x0, height, width = box
    
    y1 = y0 + height
    x1 = x0 + width
    
    x0 = clip(x0, 0, 255)
    x1 = clip(x1, 0, 255)
    y0 = clip(y0, 0, 255)
    y1 = clip(y1, 0, 255)
    
    if y0 > y1:
        y0, y1 = y1, y0
    if x0 > x1:
        x0, x1 = x1, x0

    height = y1 - y0
    width = x1 - x0
    
    box[0] = y0
    box[1] = x0
    box[2] = height
    box[3] = width

In [ ]:
predict

In [ ]:
submissions = pd.DataFrame()
submissions["ImageId"] = test_df["ImageId"]
submissions["y0"] = predict[:, 0]
submissions["x0"] = predict[:, 1]
submissions["height"] = predict[:, 2]
submissions["width"] = predict[:, 3]
submissions.head()

In [ ]:
submissions.to_csv("submission.csv", index=False)

## TODO 3: Visualize prediction

You need to visualize predicted bounding box on given test images

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
N = len(test_df)
N

In [ ]:
def draw_box(model, image_path):
    
    model = tf.keras.models.load_model(model)
    model.compile(loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(lr=config.lr)
    )
    image = cv2.imread(image_path)
    img = cv2.resize(image,(224,224))
    img = img / 256
    img = np.reshape(img,[1,224,224,3])
    predict = model.predict(img)
    predict = predict*256
    predict = predict.astype(int)
    print(predict)
    for box in predict:
      y0, x0, height, width = box
      
      y1 = y0 + height
      x1 = x0 + width
      
      x0 = clip(x0, 0, 255)
      x1 = clip(x1, 0, 255)
      y0 = clip(y0, 0, 255)
      y1 = clip(y1, 0, 255)
      
      if y0 > y1:
          y0, y1 = y1, y0
      if x0 > x1:
          x0, x1 = x1, x0

      height = y1 - y0
      width = x1 - x0
      
      box[0] = y0
      box[1] = x0
      box[2] = height
      box[3] = width
    
    
    cv2_imshow(image)		
    image_show = image.copy()
    
    x1 = 256 - x0
    x2 = x0 + height
    y1 = y0 + width
    print(x0,y0,x1,y1,x2)
    cv2.rectangle(image_show,(x1,y0),(x2,y1),(0,255,0),4)
    cv2_imshow(image_show)
    return
    

In [ ]:
draw_box(config.best_checkpoint_path,"/content/drive/My Drive/cat_face_exercise/images/06239.jpg")

In [ ]:
num_sample = 10

for idx in range(num_sample):
    idx = random.randint(0, N - 1)
    image_name = test_df["ImageId"].iloc[idx]
    image_path = os.path.join(IMAGE_DIR, image_name)
    print(image_path)
    draw_box(config.best_checkpoint_path, image_path)

## TODO 4: Customize your model

You need to build a model that can check if cat face exists in given images. If exists, then predict the cat face coordinates

In [ ]:
# TODO 4:
check_model = tf.keras.applications.DenseNet121(include_top=False,weights="imagenet",input_shape=(224,224,3))
check_model.trainable = True
model = tf.keras.Sequential([base_model,tf.keras.layers.GlobalAveragePooling2D(),tf.keras.layers.Dense(4,activation='soigmid')])

# Print out model summary
model.summary()
# nếu là mèo thì draw_box(model, image_path)
# nếu không phải mèo thì thôi